# 标注工具 — Arm + Gripper 关键帧点标注

**独立标注 notebook**，无 SAM3 / NCCL 依赖，启动快速。

## 使用流程
1. 运行 Cell 1（imports）
2. 运行 Cell 2（配置，按需修改 `TASK_NAME` / `SCENE_NAME`）
3. 运行 Cell 3（加载视频帧）
4. 运行 Cell 4（**Arm 标注 UI**）→ 标注左右臂关键帧点 → 点击 Export
5. 运行 Cell 5（**Gripper 标注 UI**）→ 标注左右夹爪关键帧点 → 点击 Export
6. 切换到 `generate_mask_airexo_data_decoupled.ipynb` 运行推理

## JSON 输出
| 文件 | 内容 |
|------|------|
| `annotation_prompts_arm_points.json` | ARM_CABLE_INITIAL_PROMPTS + ARM_CABLE_2_INITIAL_PROMPTS |
| `annotation_prompts_gripper_points.json` | GRIPPER_LEFT_KEYFRAME_PROMPTS + GRIPPER_RIGHT_KEYFRAME_PROMPTS |

In [ ]:
# ============================================================
# Imports
# （无 SAM3 / torch.distributed，启动快速）
# ============================================================
%load_ext autoreload
%autoreload 2

import json
import os
from pathlib import Path

import cv2
import matplotlib.pyplot as plt
import numpy as np

from mask_pipeline_tools import (
    get_frame_size,
    load_video_frames_for_visualization,
)
from annotation_ui_tools import (
    create_annotation_store,
    create_annotation_ui,
    load_annotation_prompts_json,
    save_annotation_prompts_json,
    seed_store_from_prompt_map,
)

plt.rcParams["axes.titlesize"] = 12
plt.rcParams["figure.titlesize"] = 12

In [ ]:
# ============================================================
# 配置 — 只需修改此区域
# ============================================================

TASK_NAME  = "task_0013"
SCENE_NAME = "scene_0003"

VIDEO_PATH = (
    f"/data/haoxiang/data/airexo2/{TASK_NAME}/train/{SCENE_NAME}"
    "/cam_105422061350/color"
)

# Object IDs（须与 generate_mask_airexo_data_decoupled.ipynb 保持一致）
ARM_OBJ_ID           = 0   # 主臂/右臂
ARM_OBJ_ID_2         = 1   # 左臂；None = 单臂模式
GRIPPER_LEFT_OBJ_ID  = 2
GRIPPER_RIGHT_OBJ_ID = 3

# JSON 输出路径（与 generate_mask_airexo_data_decoupled.ipynb 完全一致）
_scene_dir = Path(VIDEO_PATH).resolve().parent
ARM_ANNOTATION_JSON_PATH     = str(_scene_dir / "annotation_prompts_arm_points.json")
GRIPPER_ANNOTATION_JSON_PATH = str(_scene_dir / "annotation_prompts_gripper_points.json")
ANNOTATION_JSON_PATH = GRIPPER_ANNOTATION_JSON_PATH  # Gripper UI cell 使用此变量名

# 可视化
VIS_FRAME_STRIDE = 60
VIS_MAX_PLOTS    = 8

In [ ]:
# ============================================================
# 加载视频帧
# ============================================================

video_frames_for_vis = load_video_frames_for_visualization(VIDEO_PATH)
TOTAL_FRAMES = len(video_frames_for_vis)
IMG_WIDTH, IMG_HEIGHT = get_frame_size(video_frames_for_vis)

print(f"[init] frames={TOTAL_FRAMES}  size={IMG_WIDTH}x{IMG_HEIGHT}")
print(f"[init] VIDEO_PATH={VIDEO_PATH}")
print(f"[init] ARM_ANNOTATION_JSON_PATH     = {ARM_ANNOTATION_JSON_PATH}")
print(f"[init] GRIPPER_ANNOTATION_JSON_PATH = {GRIPPER_ANNOTATION_JSON_PATH}")

In [ ]:
# Arm 标注 UI
# ============================================================
# Arm 标注 UI
# 为 arm_cable（主臂/右臂, obj_id=ARM_OBJ_ID）和
#    arm_cable_2（左臂, obj_id=ARM_OBJ_ID_2）标注关键帧点
# 标注完成后点击 Export 保存到 ARM_ANNOTATION_JSON_PATH
# 若已有有效 JSON，可跳过此 cell，直接运行下方的 JSON 加载 cell
# ============================================================

ARM_ANNOTATION_OBJECT_SPECS = {
    "arm_cable": {
        "display": "Arm + Cable (主臂/右)",
        "obj_id": int(ARM_OBJ_ID),
        "target": "ARM_CABLE_INITIAL_PROMPTS",
    },
}
if ARM_OBJ_ID_2 is not None:
    ARM_ANNOTATION_OBJECT_SPECS["arm_cable_2"] = {
        "display": "Arm 2 (左臂)",
        "obj_id": int(ARM_OBJ_ID_2),
        "target": "ARM_CABLE_2_INITIAL_PROMPTS",
    }

# 若 JSON 已存在，用它 seed UI
_arm_seed_prompt_map = {
    "ARM_CABLE_INITIAL_PROMPTS": [],
    "ARM_CABLE_2_INITIAL_PROMPTS": [],
    "GRIPPER_LEFT_KEYFRAME_PROMPTS": [],
    "GRIPPER_RIGHT_KEYFRAME_PROMPTS": [],
}
if Path(ARM_ANNOTATION_JSON_PATH).exists():
    try:
        _arm_existing = load_annotation_prompts_json(
            json_path=ARM_ANNOTATION_JSON_PATH, status_prefix="[arm-annotation/seed]"
        )
        _arm_seed_prompt_map.update(_arm_existing)
        print(f"[arm-annotation] seeding UI from existing JSON: {ARM_ANNOTATION_JSON_PATH}")
    except Exception as _e:
        print(f"[arm-annotation][warn] could not seed from JSON: {_e}")

_arm_annotation_store = create_annotation_store(ARM_ANNOTATION_OBJECT_SPECS)
seed_store_from_prompt_map(
    store=_arm_annotation_store,
    object_specs=ARM_ANNOTATION_OBJECT_SPECS,
    prompt_map=_arm_seed_prompt_map,
    img_w=IMG_WIDTH,
    img_h=IMG_HEIGHT,
)

# Export 回调：补齐 schema 要求的全部键后保存
# save_json_on_export=False 原因同 gripper UI：内部保存会在补齐 ARM 字段前调用
# validate_export_prompt_map 导致 KeyError
_arm_export_result = {}

def _on_arm_export(export_prompts):
    global _arm_export_result
    _arm_export_result = {
        "ARM_CABLE_INITIAL_PROMPTS": export_prompts.get("ARM_CABLE_INITIAL_PROMPTS", []),
        "ARM_CABLE_2_INITIAL_PROMPTS": export_prompts.get("ARM_CABLE_2_INITIAL_PROMPTS", []),
        "GRIPPER_LEFT_KEYFRAME_PROMPTS": [],   # schema 要求，留空
        "GRIPPER_RIGHT_KEYFRAME_PROMPTS": [],  # schema 要求，留空
    }
    from annotation_ui_tools import save_annotation_prompts_json as _save_json
    _save_json(
        export_prompts=_arm_export_result,
        json_path=ARM_ANNOTATION_JSON_PATH,
        status_prefix="[arm-annotation]",
    )
    print(f"[arm-annotation] saved to {ARM_ANNOTATION_JSON_PATH}")

_arm_annotation_ui = create_annotation_ui(
    video_frames_for_vis=video_frames_for_vis,
    total_frames=TOTAL_FRAMES,
    img_width=IMG_WIDTH,
    img_height=IMG_HEIGHT,
    object_specs=ARM_ANNOTATION_OBJECT_SPECS,
    annotation_store=_arm_annotation_store,
    on_export=_on_arm_export,
    auto_display=True,
    status_prefix="[arm-annotation]",
    export_json_path=ARM_ANNOTATION_JSON_PATH,
    save_json_on_export=False,
)

if not _arm_annotation_ui.get("widget_ready", False):
    print(f"[arm-annotation][fallback] Widget not available: {_arm_annotation_ui.get('widget_error')}")
    print(f"[arm-annotation][fallback] 请直接编辑 JSON 文件: {ARM_ANNOTATION_JSON_PATH}")

In [ ]:
# Gripper 标注 UI

# ============================================================
# Gripper 标注 UI
# 在此处为 gripper_left / gripper_right 添加标注点，然后点击 Export 保存 JSON
# 若已有有效 JSON，可跳过此 cell，直接运行下方的 JSON 加载 cell
# ============================================================

# 仅含 gripper 对象，不含 arm（arm 无需标注）
GRIPPER_ANNOTATION_OBJECT_SPECS = {
    "gripper_left": {
        "display": "Gripper Left",
        "obj_id": int(GRIPPER_LEFT_OBJ_ID),
        "target": "GRIPPER_LEFT_KEYFRAME_PROMPTS",
    },
    "gripper_right": {
        "display": "Gripper Right",
        "obj_id": int(GRIPPER_RIGHT_OBJ_ID),
        "target": "GRIPPER_RIGHT_KEYFRAME_PROMPTS",
    },
}

# 若 JSON 已存在，用它 seed UI（不强制加载，仅用于填充初始状态）
_seed_prompt_map = {
    "ARM_CABLE_INITIAL_PROMPTS": [],
    "GRIPPER_LEFT_KEYFRAME_PROMPTS": [],
    "GRIPPER_RIGHT_KEYFRAME_PROMPTS": [],
}
if Path(ANNOTATION_JSON_PATH).exists():
    try:
        _existing = load_annotation_prompts_json(
            json_path=ANNOTATION_JSON_PATH, status_prefix="[annotation/seed]"
        )
        _seed_prompt_map.update(_existing)
        print(f"[annotation] seeding UI from existing JSON: {ANNOTATION_JSON_PATH}")
    except Exception as _e:
        print(f"[annotation][warn] could not seed from JSON: {_e}")

_annotation_store = create_annotation_store(GRIPPER_ANNOTATION_OBJECT_SPECS)
seed_store_from_prompt_map(
    store=_annotation_store,
    object_specs=GRIPPER_ANNOTATION_OBJECT_SPECS,
    prompt_map=_seed_prompt_map,
    img_w=IMG_WIDTH,
    img_h=IMG_HEIGHT,
)

# Export 回调：补齐 ARM_CABLE_INITIAL_PROMPTS 字段后自行保存 JSON。
# 注意：必须用 save_json_on_export=False 禁用 create_annotation_ui 的内部保存，
# 因为内部保存会在添加 ARM 字段之前就调用 validate_export_prompt_map，导致 KeyError。
_gripper_export_result = {}

def _on_gripper_export(export_prompts):
    global _gripper_export_result
    # 补齐 ARM_CABLE_INITIAL_PROMPTS，满足 JSON schema 要求
    _gripper_export_result = {
        "ARM_CABLE_INITIAL_PROMPTS": [],
        "GRIPPER_LEFT_KEYFRAME_PROMPTS": export_prompts.get("GRIPPER_LEFT_KEYFRAME_PROMPTS", []),
        "GRIPPER_RIGHT_KEYFRAME_PROMPTS": export_prompts.get("GRIPPER_RIGHT_KEYFRAME_PROMPTS", []),
    }
    from annotation_ui_tools import save_annotation_prompts_json as _save_json
    _save_json(
        export_prompts=_gripper_export_result,
        json_path=ANNOTATION_JSON_PATH,
        status_prefix="[annotation/gripper]",
    )
    print(f"[annotation] saved to {ANNOTATION_JSON_PATH}")

_annotation_ui = create_annotation_ui(
    video_frames_for_vis=video_frames_for_vis,
    total_frames=TOTAL_FRAMES,
    img_width=IMG_WIDTH,
    img_height=IMG_HEIGHT,
    object_specs=GRIPPER_ANNOTATION_OBJECT_SPECS,
    annotation_store=_annotation_store,
    on_export=_on_gripper_export,
    auto_display=True,
    status_prefix="[annotation/gripper]",
    export_json_path=ANNOTATION_JSON_PATH,
    save_json_on_export=False,  # 禁用内部保存，由 _on_gripper_export 回调负责
)

if not _annotation_ui.get("widget_ready", False):
    print(f"[annotation][fallback] Widget not available: {_annotation_ui.get('widget_error')}")
    print(f"[annotation][fallback] 请直接编辑 JSON 文件: {ANNOTATION_JSON_PATH}")